# Tokenizer
 - 입력된 문장들을 토큰이라는 작은 unit들로 나누어주는 역할을 한다.
 - 토큰들 숫자데이터로 변환해 NLP task에 사용한다.
 - 문자기반, 단어기반, 서브워드 토크나이저 등등

### 1. Word Tokenizer
 - 단어를 기준으로 토큰화를 하는 토크나이저
 - ex) "맑은 하늘에 햇빛이 비쳐요." => '맑은', '하늘에', '햇빛이', '비쳐요', '.'
 - 단점으로, 어미 변화로 인해 유사단어가 많아질 수 있고 단어를 표현하는 벡터가 다른 의미를 가질 수 있다.
 - '하늘에', '하늘을', 하늘에서', '하늘이' 등
 - 사전의 개수가 매우 많이질 수 있다.
 
### 2. Character Tokenizer
 - 개별 문자를 기준으로 토큰화를 하는 토크나이저
 - ex) "안녕하세요" => '안', '녕', '하', '세', '요'
 - OOV 발생 가능성이 낮다.
 - 단점으로 각 글자 '안', '녕', '하', '세', '요'의 벡터가 '안녕하세요'를 표현하지 못할 수 있다.

### 3. Subword Tokenizer
 - 단어를 나누어 단어안에 단어(subword)들로 토큰화하는 토크나이저
 - 빈번하게 사용되는 단어는 분할하지 않고 드물게 등장하는 단어를 의미있는 하위 단어로 분할한다.
 - ex) '경찰차', '경찰청', '경찰관' => '경찰', '#차', '#청', '#관'
 - OOV 문제를 완화
 - BPE, SentencePiece, WordPiece 등 여러 방법이 존재한다.

In [4]:
# BPE 
# Neural Machine Translation of Rare Words with Subword Units
# https://arxiv.org/abs/1508.07909

import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {
    'l o w </w>' : 5,
    'l o w e r </w>' : 2,
    'n e w e s t </w>':6,
    'w i d e s t </w>':3
}
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


In [5]:
# SentencePiece
!pip install sentencepiece

In [7]:
# SentencePiece
import sentencepiece as spm

corpus = "data/chatbot_data.txt"
prefix = "CBot_vocab"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" +    # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +    # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +    # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +    # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +    # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]")    # 사용자 정의 토큰